# Data Wrangling:

In [35]:
#import relevent packages
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
#set api_key var
api_key = 'cRs2AAh8gcVCYm0BOD9lQRo6Bj9J11dKKeiVkZQB'

In [37]:
def get_bills(congress):
    
    bills_final = pd.DataFrame()
    
    index = 0
    returns = 20

    while returns == 20:
    
        offset = 20*index
    
        #query database and convert to DataFrame
        response = requests.get('https://api.propublica.org/congress/v1/' + str(congress) +'/house/bills/introduced.json', 
                                {'offset': offset}, headers = {'X-API-Key': api_key})
    
        try:
            response_body = response.json()
        except:
            break
        try:    
            bills = response_body['results'][0]['bills']
        except:
            break
        df = pd.DataFrame(bills)
        returns = df.shape[0]
    
        #filter out resolutions and ammendments
        df = df[df.bill_type == 'hr'] 
    
        #append new data
        bills_final = pd.concat([bills_final, df])
        
        if index % 10 == 0:
            print(index)
        index += 1
    print('Done')
    keep_cols = ['bill_id', 'bill_slug', 'title', 'short_title', 'sponsor_title', 'sponsor_id',
             'sponsor_name', 'sponsor_party', 'introduced_date', 'active', 'last_vote', 'house_passage',
            'senate_passage', 'enacted', 'vetoed', 'cosponsors', 'cosponsors_by_party',
            'committee_codes', 'subcommittee_codes', 'primary_subject', 'summary', 'summary_short', 
            'latest_major_action_date', 'latest_major_action']
    bills_final = bills_final[keep_cols]
    print(bills_final.shape)
    
    return bills_final


In [38]:
def get_members(bill_df):

    members = pd.DataFrame()
    congress = str(bill_df.congress.unique()[0])
    print(congress)
    print(type(congress))
    #iterate through sponsors
    for member in bill_df.sponsor_id.unique():
        #get general member data (id, gender)
        response = requests.get("https://api.propublica.org/congress/v1/members/" + str(member) + ".json", 
                                headers = {'X-API-Key': api_key})
        response_body = response.json()
        member_data = response_body['results'][0]
        member_data = pd.DataFrame(member_data)
        df = pd.DataFrame(columns = ['committee_codes', 'committee_side', 'committee_rank_in_party', 
                                     'subcommittee_codes', 'subcommittee_side', 'subcommittee_rank_in_party'])
        
        #match subset of member data to congress of interest
        congresses = []
        for i in range(member_data.shape[0]):
            congresses.append(str(member_data.roles[i]['congress']))
        index = [i for i, x in enumerate(congresses) if x == congress][0]
        
        #get leadership_role, seniority, missed_votes_pct, votes_with_party_pct, votes_against_party_pct,
        #committee_codes, committee_side, committee_rank_in_party, subcommittee_codes, subcommittee_side,
        #subcommittee_rank_in_party
        df['id'] = member_data['id'].unique()
        df['gender'] = member_data['gender'].unique()
        
        df['leadership_role'] = member_data.roles[index]['leadership_role']
        df['seniority'] = member_data.roles[index]['seniority']
        df['missed_votes_pct'] = member_data.roles[index]['missed_votes_pct']
        df['votes_with_party_pct'] = member_data.roles[index]['votes_with_party_pct']
        df['votes_against_party_pct'] = member_data.roles[index]['votes_against_party_pct']
        
        df['committee_codes'][0] = [member_data.roles[index]['committees'][i]['code'] for i in range(len(member_data.roles[index]['committees']))]
        
        try:
            df['committee_side'][0] = [member_data.roles[index]['committees'][i]['side'] for i in range(len(member_data.roles[index]['committees']))]
        except:
            df['committee_side'][0] = np.nan
            
        df['committee_rank_in_party'][0] = [member_data.roles[index]['committees'][i]['rank_in_party'] for i in range(len(member_data.roles[index]['committees']))]
        df['subcommittee_codes'][0] = [member_data.roles[index]['subcommittees'][i]['code'] for i in range(len(member_data.roles[index]['subcommittees']))]
       
        try:
            df['subcommittee_side'][0] = [member_data.roles[index]['subcommittees'][i]['side'] for i in range(len(member_data.roles[index]['subcommittees']))]
        except:
            df['subcommittee_side'][0] = np.nan
            
        df['subcommittee_rank_in_party'][0] = [member_data.roles[index]['subcommittees'][i]['rank_in_party'] for i in range(len(member_data.roles[index]['subcommittees']))]

        members = pd.concat([members, df])
        if members.shape[0] % 10 == 0:
            print(members.shape[0])
    
    print(members.shape)
    return members


In [39]:
def merge_on_id(bills, members):
    data = bills.merge(members, how= 'left', left_on = 'sponsor_id', right_on = 'id')
    
    return(data)

In [ ]:
bills113 = get_bills(113)
bills113['congress'] = str(113)

In [ ]:
bills113.to_csv(r'bills113.csv', index = False)

In [40]:
bills113 = pd.read_csv('bills113.csv')
bills113.head()

,bill_id,bill_slug,title,short_title,sponsor_title,sponsor_id,sponsor_name,sponsor_party,introduced_date,active,...,cosponsors,cosponsors_by_party,committee_codes,subcommittee_codes,primary_subject,summary,summary_short,latest_major_action_date,latest_major_action,congress
0,hr5893-113,hr5893,To restore the First Amendment Rights of Photo...,Ansel Adams Act,Rep.,S000937,Steve Stockman,R,2015-01-02,False,...,0,{},"['HSJU', 'HSAG', 'HSII', 'HSGO']",[],Government Operations and Politics,Ansel Adams Act - Declares that it is contrary...,Ansel Adams Act - Declares that it is contrary...,2015-01-02,Referred to House Judiciary,113
1,hr5892-113,hr5892,To protect cryptocurrencies.,Online Market Protection Act of 2014,Rep.,S000937,Steve Stockman,R,2015-01-02,False,...,0,{},"['HSAG', 'HSWM', 'HSBA']",[],Finance and Financial Sector,Online Market Protection Act of 2014 - Prohibi...,Online Market Protection Act of 2014 - Prohibi...,2015-01-02,Referred to House Agriculture,113
2,hr5891-113,hr5891,To increase the standards of school lunch prog...,Michelle Obama Nutritional Act,Rep.,S000937,Steve Stockman,R,2015-01-02,False,...,0,{},['HSED'],[],Agriculture and Food,Michelle Obama Nutritional Act - Requires the ...,Michelle Obama Nutritional Act - Requires the ...,2015-01-02,Referred to the House Committee on Education a...,113
3,hr5890-113,hr5890,To conduct a study to ensure that enhanced com...,No Missing Planes Act,Rep.,J000032,Sheila Jackson Lee,D,2015-01-02,False,...,0,{},['HSPW'],['HSPW05'],Transportation and Public Works,No Missing Planes Act - Directs the Administra...,No Missing Planes Act - Directs the Administra...,2015-01-02,Referred to the House Committee on Transportat...,113
4,hr5889-113,hr5889,"To revise the Uniform Crime Reports, and the N...",Justifiable Homicide Accuracy in Reporting Act...,Rep.,J000032,Sheila Jackson Lee,D,2015-01-02,False,...,0,{},['HSJU'],[],Crime and Law Enforcement,Justifiable Homicide Accuracy in Reporting Act...,Justifiable Homicide Accuracy in Reporting Act...,2015-01-02,Referred to the House Committee on the Judiciary.,113


In [ ]:
bills114 = get_bills(114)
bills114['congress'] = str(114)

In [ ]:
bills114.to_csv(r'bills114.csv', index = False)

In [41]:
bills114 = pd.read_csv('bills114.csv')
bills114.head()

,bill_id,bill_slug,title,short_title,sponsor_title,sponsor_id,sponsor_name,sponsor_party,introduced_date,active,...,cosponsors,cosponsors_by_party,committee_codes,subcommittee_codes,primary_subject,summary,summary_short,latest_major_action_date,latest_major_action,congress
0,hr6536-114,hr6536,To prohibit the employment of school bus drive...,No School Bus Drivers with DUIs Act of 2016,Rep.,M001188,Grace Meng,D,2016-12-30,False,...,0,{},['HSED'],[],Education,No School Bus Drivers with DUIs Act of 2016 Th...,No School Bus Drivers with DUIs Act of 2016 Th...,2016-12-30,Referred to the House Committee on Education a...,114
1,hr6535-114,hr6535,To require the President to obtain written app...,Nuclear Sanity Act,Rep.,G000556,Alan Grayson,D,2016-12-27,False,...,0,{},"['HSAS', 'HSFA']",[],International Affairs,Nuclear Sanity Act This bill requires the Pres...,Nuclear Sanity Act This bill requires the Pres...,2016-12-27,"Referred to the Committee on Foreign Affairs, ...",114
2,hr6534-114,hr6534,To require the creation and maintenance of a M...,MadeInAmerica.gov Act,Rep.,M001188,Grace Meng,D,2016-12-20,False,...,0,{},['HSIF'],[],Commerce,MadeInAmerica.gov Act This bill requires the D...,MadeInAmerica.gov Act This bill requires the D...,2016-12-20,Referred to the House Committee on Energy and ...,114
3,hr6531-114,hr6531,To authorize the programs of the National Aero...,National Aeronautics and Space Administration ...,Rep.,S000583,Lamar Smith,R,2016-12-20,False,...,0,{},['HSSY'],[],"Science, Technology, Communications",National Aeronautics and Space Administration ...,National Aeronautics and Space Administration ...,2016-12-20,"Referred to the House Committee on Science, Sp...",114
4,hr6532-114,hr6532,To provide for the adjustment of status of cer...,Liberian Refugee Immigration Fairness Act of 2016,Rep.,E000288,Keith Ellison,D,2016-12-20,False,...,2,{'D': 2},['HSJU'],[],Immigration,Liberian Refugee Immigration Fairness Act of 2...,Liberian Refugee Immigration Fairness Act of 2...,2016-12-20,Referred to the House Committee on the Judiciary.,114


In [8]:
bills115 = get_bills(115)
bills115['congress'] = str(115)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
Done
(7394, 24)


In [9]:
bills115.to_csv(r'bills115.csv', index = False)

In [42]:
bills115 = pd.read_csv('bills115.csv')
bills115.head()

,bill_id,bill_slug,title,short_title,sponsor_title,sponsor_id,sponsor_name,sponsor_party,introduced_date,active,...,cosponsors,cosponsors_by_party,committee_codes,subcommittee_codes,primary_subject,summary,summary_short,latest_major_action_date,latest_major_action,congress
0,hr7401-115,hr7401,To modify provisions of law relating to refuge...,Strengthening Refugee Resettlement Act,Rep.,E000288,Keith Ellison,D,2019-01-02,False,...,0,{},['HSJU'],[],Immigration,NaN,NaN,2019-01-02,Referred to the Subcommittee on Trade.,115
1,hr7400-115,hr7400,Making continuing appropriations for the Coast...,Making continuing appropriations for the Coast...,Rep.,W000826,Susan Wild,D,2019-01-02,False,...,0,{},['HSAP'],[],Transportation and Public Works,NaN,NaN,2019-01-02,Referred to the House Committee on Appropriati...,115
2,hr7399-115,hr7399,To amend the Federal Election Campaign Act of ...,Inaugural Fund Integrity Act,Rep.,S001205,Mary Gay Scanlon,D,2018-12-27,False,...,0,{},['HSHA'],[],Government Operations and Politics,NaN,NaN,2018-12-27,Referred to the Committee on House Administrat...,115
3,hr7397-115,hr7397,To provide further additional continuing appro...,To provide further additional continuing appro...,Rep.,H000874,Steny H. Hoyer,D,2018-12-22,False,...,0,{},['HSAP'],[],Economics and Public Finance,DIVISION A--FURTHER ADDITIONAL CONTINUING APPR...,DIVISION A--FURTHER ADDITIONAL CONTINUING APPR...,2018-12-22,"Referred to the Committee on Appropriations, a...",115
4,hr7398-115,hr7398,To prohibit the operation of an exercise facil...,SPA Act,Rep.,F000454,Bill Foster,D,2018-12-22,False,...,0,{},['HSHA'],[],Congress,NaN,NaN,2018-12-22,Referred to the House Committee on House Admin...,115


In [43]:
members113 = get_members(bills113)

113
<class 'str'>


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44

10
20
30
40
50
60
70
80
90
100


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
(440, 13)


In [44]:
members113.to_csv(r'members113.csv', index = False)

In [51]:
members113 = pd.read_csv('members113.csv')
members113.head()

,committee_codes,committee_side,committee_rank_in_party,subcommittee_codes,subcommittee_side,subcommittee_rank_in_party,id,gender,leadership_role,seniority,missed_votes_pct,votes_with_party_pct,votes_against_party_pct
0,"['HSFA', 'HSSY']",NaN,"[19, 13]",[],[],[],S000937,M,NaN,2,5.90,85.58,14.34
1,"['HSHM', 'HSJU']",NaN,"[3, 5]",[],[],[],J000032,F,NaN,20,7.48,93.85,6.06
2,['HSBA'],NaN,[13],[],[],[],C001061,M,NaN,10,10.22,95.99,3.91
3,"['HSSY', 'HSAS']",NaN,"[12, 21]",[],[],[],K000381,M,NaN,2,1.16,91.53,8.39
4,"['HSII', 'HSPW']",NaN,"[2, 2]",[],[],[],Y000033,M,NaN,42,7.89,88.86,11.05


In [52]:
members114 = get_members(bills114)

114
<class 'str'>


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
(443, 13)


In [53]:
members114.to_csv(r'members114.csv', index = False)

In [54]:
members114 = pd.read_csv('members114.csv')
members114.head()

,committee_codes,committee_side,committee_rank_in_party,subcommittee_codes,subcommittee_side,subcommittee_rank_in_party,id,gender,leadership_role,seniority,missed_votes_pct,votes_with_party_pct,votes_against_party_pct
0,"['HSFA', 'HSSM']",NaN,"[14, 5]",[],[],[],M001188,F,NaN,4,4.55,97.36,2.56
1,"['HSFA', 'HSSY']",NaN,"[11, 7]",[],[],[],G000556,M,NaN,4,4.08,94.76,5.08
2,"['HSHM', 'HSJU', 'HSSY']",NaN,"[2, 3, 1]",[],[],[],S000583,M,NaN,30,1.96,97.52,2.40
3,['HSBA'],NaN,[14],[],[],[],E000288,M,NaN,10,4.53,96.73,3.11
4,"['HSFA', 'HSIF']",NaN,"[1, 4]",[],[],[],E000179,M,NaN,28,7.12,96.87,3.05


In [55]:
members115 = get_members(bills115)

115
<class 'str'>


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
(447, 13)


In [56]:
members115.to_csv(r'members115.csv', index = False)

In [57]:
members115 = pd.read_csv('members115.csv')
members115.head()

,committee_codes,committee_side,committee_rank_in_party,subcommittee_codes,subcommittee_side,subcommittee_rank_in_party,id,gender,leadership_role,seniority,missed_votes_pct,votes_with_party_pct,votes_against_party_pct
0,['HSBA'],['minority'],[13],"['HSBA09', 'HSBA15', 'HSBA16']","['minority', 'minority', 'minority']","[2, 7, 6]",E000288,M,NaN,12,17.34,94.66,5.34
1,[],[],[],[],[],[],W000826,F,NaN,2,12.66,100.00,0.00
2,[],[],[],[],[],[],S001205,F,NaN,2,1.19,98.78,1.22
3,[],[],[],[],[],[],H000874,M,Minority Whip,38,5.04,98.39,1.61
4,"['HSBA', 'HSSY']","['minority', 'minority']","[16, 13]","['HSBA01', 'HSBA16', 'HSBA20', 'HSSY16', 'HSSY...","['minority', 'minority', 'minority', 'minority...","[4, 7, 3, 8, 7]",F000454,M,NaN,6,2.15,96.10,3.90


In [58]:
data113 = merge_on_id(bills113, members113)

In [59]:
data113.to_csv(r'data113.csv', index = False)

In [60]:
data113 = pd.read_csv('data113.csv')
data113.head()

,bill_id,bill_slug,title,short_title,sponsor_title,sponsor_id,sponsor_name,sponsor_party,introduced_date,active,...,subcommittee_codes_y,subcommittee_side,subcommittee_rank_in_party,id,gender,leadership_role,seniority,missed_votes_pct,votes_with_party_pct,votes_against_party_pct
0,hr5893-113,hr5893,To restore the First Amendment Rights of Photo...,Ansel Adams Act,Rep.,S000937,Steve Stockman,R,2015-01-02,False,...,[],[],[],S000937,M,NaN,2,5.90,85.58,14.34
1,hr5892-113,hr5892,To protect cryptocurrencies.,Online Market Protection Act of 2014,Rep.,S000937,Steve Stockman,R,2015-01-02,False,...,[],[],[],S000937,M,NaN,2,5.90,85.58,14.34
2,hr5891-113,hr5891,To increase the standards of school lunch prog...,Michelle Obama Nutritional Act,Rep.,S000937,Steve Stockman,R,2015-01-02,False,...,[],[],[],S000937,M,NaN,2,5.90,85.58,14.34
3,hr5890-113,hr5890,To conduct a study to ensure that enhanced com...,No Missing Planes Act,Rep.,J000032,Sheila Jackson Lee,D,2015-01-02,False,...,[],[],[],J000032,F,NaN,20,7.48,93.85,6.06
4,hr5889-113,hr5889,"To revise the Uniform Crime Reports, and the N...",Justifiable Homicide Accuracy in Reporting Act...,Rep.,J000032,Sheila Jackson Lee,D,2015-01-02,False,...,[],[],[],J000032,F,NaN,20,7.48,93.85,6.06


In [61]:
data114 = merge_on_id(bills114, members114)

In [62]:
data114.to_csv(r'data114.csv', index = False)

In [63]:
data114 = pd.read_csv('data114.csv')
data114.head()

,bill_id,bill_slug,title,short_title,sponsor_title,sponsor_id,sponsor_name,sponsor_party,introduced_date,active,...,subcommittee_codes_y,subcommittee_side,subcommittee_rank_in_party,id,gender,leadership_role,seniority,missed_votes_pct,votes_with_party_pct,votes_against_party_pct
0,hr6536-114,hr6536,To prohibit the employment of school bus drive...,No School Bus Drivers with DUIs Act of 2016,Rep.,M001188,Grace Meng,D,2016-12-30,False,...,[],[],[],M001188,F,NaN,4,4.55,97.36,2.56
1,hr6535-114,hr6535,To require the President to obtain written app...,Nuclear Sanity Act,Rep.,G000556,Alan Grayson,D,2016-12-27,False,...,[],[],[],G000556,M,NaN,4,4.08,94.76,5.08
2,hr6534-114,hr6534,To require the creation and maintenance of a M...,MadeInAmerica.gov Act,Rep.,M001188,Grace Meng,D,2016-12-20,False,...,[],[],[],M001188,F,NaN,4,4.55,97.36,2.56
3,hr6531-114,hr6531,To authorize the programs of the National Aero...,National Aeronautics and Space Administration ...,Rep.,S000583,Lamar Smith,R,2016-12-20,False,...,[],[],[],S000583,M,NaN,30,1.96,97.52,2.40
4,hr6532-114,hr6532,To provide for the adjustment of status of cer...,Liberian Refugee Immigration Fairness Act of 2016,Rep.,E000288,Keith Ellison,D,2016-12-20,False,...,[],[],[],E000288,M,NaN,10,4.53,96.73,3.11


In [64]:
data115 = merge_on_id(bills115, members115)

In [65]:
data115.to_csv(r'data115.csv', index = False)

In [66]:
data115 = pd.read_csv('data115.csv')
data115.head()

,bill_id,bill_slug,title,short_title,sponsor_title,sponsor_id,sponsor_name,sponsor_party,introduced_date,active,...,subcommittee_codes_y,subcommittee_side,subcommittee_rank_in_party,id,gender,leadership_role,seniority,missed_votes_pct,votes_with_party_pct,votes_against_party_pct
0,hr7401-115,hr7401,To modify provisions of law relating to refuge...,Strengthening Refugee Resettlement Act,Rep.,E000288,Keith Ellison,D,2019-01-02,False,...,"['HSBA09', 'HSBA15', 'HSBA16']","['minority', 'minority', 'minority']","[2, 7, 6]",E000288,M,NaN,12,17.34,94.66,5.34
1,hr7400-115,hr7400,Making continuing appropriations for the Coast...,Making continuing appropriations for the Coast...,Rep.,W000826,Susan Wild,D,2019-01-02,False,...,[],[],[],W000826,F,NaN,2,12.66,100.00,0.00
2,hr7399-115,hr7399,To amend the Federal Election Campaign Act of ...,Inaugural Fund Integrity Act,Rep.,S001205,Mary Gay Scanlon,D,2018-12-27,False,...,[],[],[],S001205,F,NaN,2,1.19,98.78,1.22
3,hr7397-115,hr7397,To provide further additional continuing appro...,To provide further additional continuing appro...,Rep.,H000874,Steny H. Hoyer,D,2018-12-22,False,...,[],[],[],H000874,M,Minority Whip,38,5.04,98.39,1.61
4,hr7398-115,hr7398,To prohibit the operation of an exercise facil...,SPA Act,Rep.,F000454,Bill Foster,D,2018-12-22,False,...,"['HSBA01', 'HSBA16', 'HSBA20', 'HSSY16', 'HSSY...","['minority', 'minority', 'minority', 'minority...","[4, 7, 3, 8, 7]",F000454,M,NaN,6,2.15,96.10,3.90
